In [2]:
import requests, csv 
import us
import pandas as pd
import time
from easymoney.money import EasyPeasy
import json

/Users/gabrielmelmed/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# I. Downloading the variables table

i. The first step is to the dictionary table for the ACS's variables and save it to a csv. The aim is to create a 'relational database' with a separate tables for values.

In [5]:
vars_df = pd.read_csv('census_vars.csv')

In [6]:
# change !! into a space in the label col
vars_df['label'] = vars_df['label'].str.replace('!!', ' ')

In [7]:
vars_df = vars_df[['index', 'label']]
vars_df

,index,label
0,for,Census API FIPS 'for' clause
1,in,Census API FIPS 'in' clause
2,ucgid,Uniform Census Geography Identifier clause
3,S0804_C04_068E,Estimate Public transportation (excluding taxi...
4,S0503_C02_078E,Estimate Foreign born; Born in Europe Civilian...
...,...,...
18822,S2402_C02_035E,"Estimate Male Full-time, year-round civilian e..."
18823,S1002_C04_004E,Estimate 60 years and over Percent distributio...
18824,S0601_C02_009E,Estimate Native; born in state of residence To...
18825,S2411_C01_012E,Estimate Median earnings (dollars) Civilian em...


In [ ]:
# save as a csv
# vars_df.to_csv("census_vars.csv", index=False)

In [ ]:
# find the variables that contain the word "rent"
rent_vars = vars_df[vars_df['label'].str.contains('rent', case=False)]

rent_vars

In [ ]:
income_vars = vars_df[vars_df['label'].str.contains('income', case=False) & vars_df['label'].str.contains('mean', case=False)]
income_vars

,index,label
285,S0506_C01_103E,Estimate Total INCOME IN THE PAST 12 MONTHS (I...
295,S0506_C01_105E,Estimate Total INCOME IN THE PAST 12 MONTHS (I...
302,S0506_C01_107E,Estimate Total INCOME IN THE PAST 12 MONTHS (I...
419,S0502_C01_108E,Estimate Total INCOME IN THE PAST 12 MONTHS (I...
435,S0502_C01_106E,Estimate Total INCOME IN THE PAST 12 MONTHS (I...
...,...,...
17800,S0503_C03_099E,Estimate Foreign born; Born in Northern and We...
18316,S2601A_C04_102E,Estimate Noninstitutionalized group quarters p...
18332,S2601A_C04_101E,Estimate Noninstitutionalized group quarters p...
18739,S0506_C05_099E,Estimate Foreign born; Born in Caribbean INCOM...


In [8]:
pop_vars = vars_df[vars_df['label'].str.contains('population', case=False)]
pop_vars

,index,label
4,S0503_C02_078E,Estimate Foreign born; Born in Europe Civilian...
5,S2603_C07_076E,Estimate Military quarters/military ships WORL...
6,S0701PR_C01_028E,Estimate Total MARITAL STATUS Population 15 ye...
8,S0503_C02_077E,Estimate Foreign born; Born in Europe Civilian...
9,S2603_C07_075E,Estimate Military quarters/military ships PLAC...
...,...,...
18819,S0601_C02_008E,Estimate Native; born in state of residence To...
18820,S2411_C01_011E,Estimate Median earnings (dollars) Civilian em...
18822,S2402_C02_035E,"Estimate Male Full-time, year-round civilian e..."
18824,S0601_C02_009E,Estimate Native; born in state of residence To...


In [ ]:
pip install census


SyntaxError: invalid syntax (1553123308.py, line 1)

In [11]:
pip install us

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
api_key  = "9330018ae4a2bd20da22ced1281145d9405dc349" # replace with your API key

ii. Store the values we want in a list

In [ ]:
vars_of_interest = [
    "S1903_C03_015E", # 'Estimate Median income (dollars) FAMILIES Families'
    "S1903_C03_001E", # 'Estimate Mean income (dollars) HOUSEHOLD INCOME All households'
    
]

In [21]:
dataset = 'acs/acs5' # American Community Survey 5-year data
series = "B21004_001E" # Median Income


responses = [] 
for year in list(range(2010, 2020))+[2022]: # Loop through years, there's no data for 2020
    data_url = f'https://api.census.gov/data/{year}/{dataset}?get=NAME,{series}&for=county:*&in=state:*&key={api_key}'
    print(f'{year}: {data_url}')
    df = pd.read_json(data_url) # Read data from API into a pandas dataframe
    df.columns = df.iloc[0] # Set first row as column names
    df = df[1:] # Remove first row
    df["Year"] = year
    time.sleep(1) # Sleep for 1 second to avoid hitting the API too fast
    responses.append(df)

2010: https://api.census.gov/data/2010/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2011: https://api.census.gov/data/2011/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2012: https://api.census.gov/data/2012/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2013: https://api.census.gov/data/2013/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2014: https://api.census.gov/data/2014/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2015: https://api.census.gov/data/2015/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2016: https://api.census.gov/data/2016/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349
2017: https://api.census.gov/data/

In [17]:
url=f"https://api.census.gov/data/2022/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key={api_key}"

response = requests.get(url)

d = response.json()

In [20]:
url

'https://api.census.gov/data/2022/acs/acs5?get=NAME,B21004_001E&for=county:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349'

In [ ]:
https://api.census.gov/data/2010/acs/acs5?get=NAME,B21004_001E&for=country:*&in=state:*&key=9330018ae4a2bd20da22ced1281145d9405dc349

### Population

Population is easier. We can just use static download links 

In [1]:
county_pop_url = "//www2.census.gov/programs-surveys/popest/tables/2020-2023/counties/totals/co-est2023-pop.xlsx"



In [2]:
import pandas as pd

In [3]:
pops = pd.read_excel(county_pop_url, skiprows=3)

FileNotFoundError: [Errno 2] No such file or directory: '//www2.census.gov/programs-surveys/popest/tables/2020-2023/counties/totals/co-est2023-pop.xlsx'

## Using Census Package 

In [15]:
# import necessary libraries
import census
import us
from typing import Dict, List
import pandas as pd

In [16]:
# Define the API key
c = census.Census(api_key)

In [8]:
# Define the API key
c = census.Census(api_key)
# Get population estimates for all states
state_pops = c.acs5.get('B01003_001E', {'for': 'state:*'}, year=2022)
state_pops

/Users/gabrielmelmed/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[{'B01003_001E': 5028092.0, 'state': '01'},
 {'B01003_001E': 734821.0, 'state': '02'},
 {'B01003_001E': 7172282.0, 'state': '04'},
 {'B01003_001E': 3018669.0, 'state': '05'},
 {'B01003_001E': 39356104.0, 'state': '06'},
 {'B01003_001E': 5770790.0, 'state': '08'},
 {'B01003_001E': 3611317.0, 'state': '09'},
 {'B01003_001E': 993635.0, 'state': '10'},
 {'B01003_001E': 670587.0, 'state': '11'},
 {'B01003_001E': 21634529.0, 'state': '12'},
 {'B01003_001E': 10722325.0, 'state': '13'},
 {'B01003_001E': 1450589.0, 'state': '15'},
 {'B01003_001E': 1854109.0, 'state': '16'},
 {'B01003_001E': 12757634.0, 'state': '17'},
 {'B01003_001E': 6784403.0, 'state': '18'},
 {'B01003_001E': 3188836.0, 'state': '19'},
 {'B01003_001E': 2935922.0, 'state': '20'},
 {'B01003_001E': 4502935.0, 'state': '21'},
 {'B01003_001E': 4640546.0, 'state': '22'},
 {'B01003_001E': 1366949.0, 'state': '23'},
 {'B01003_001E': 6161707.0, 'state': '24'},
 {'B01003_001E': 6984205.0, 'state': '25'},
 {'B01003_001E': 10057921.0, 's

In [9]:
# convert to a dataframe
state_pops_df = pd.DataFrame(state_pops)
state_pops_df.columns = ['population', 'id']

In [ ]:
state_pops_df

In [10]:
# get population for all counties
county_pops = c.acs5.get('B01003_001E', {'for': 'county:*'}, year=2022)

In [45]:
county_pops_df.dtypes

B01003_001E    float64
state           object
county          object
dtype: object

In [11]:
# convert to a dataframe
county_pops_df = pd.DataFrame(county_pops)
# convert state and county to int
county_pops_df['state'] = county_pops_df['state'].astype(int)
county_pops_df['county'] = county_pops_df['county'].astype(int)
# create a fips column
county_pops_df['fips'] = county_pops_df['state']*1000 + county_pops_df['county']
# county_pops_df = county_pops_df.drop(columns=['state', 'county'])
# county_pops_df.columns = ['population', 'fips']
county_pops_df

,B01003_001E,state,county,fips
0,58761.0,1,1,1001
1,233420.0,1,3,1003
2,24877.0,1,5,1005
3,22251.0,1,7,1007
4,59077.0,1,9,1009
...,...,...,...,...
3217,54182.0,72,145,72145
3218,8199.0,72,147,72147
3219,21984.0,72,149,72149
3220,30313.0,72,151,72151


In [14]:
county_pops_df[county_pops_df['state'] == 9]

,B01003_001E,state,county,fips
309,977165.0,9,110,9110
310,326381.0,9,120,9120
311,175244.0,9,130,9130
312,451887.0,9,140,9140
313,95687.0,9,150,9150
314,112696.0,9,160,9160
315,571298.0,9,170,9170
316,280293.0,9,180,9180
317,620666.0,9,190,9190


In [42]:
# get tables for sf1
c.sf1.tables()
# turn it into a dataframe
tables = pd.DataFrame(c.sf1.tables())
tables

,name,description,variables
0,P38B,FAMILY TYPE BY PRESENCE AND AGE OF OWN CHILDRE...,http://api.census.gov/data/2010/dec/sf1/groups...
1,P38A,FAMILY TYPE BY PRESENCE AND AGE OF OWN CHILDRE...,http://api.census.gov/data/2010/dec/sf1/groups...
2,P38D,FAMILY TYPE BY PRESENCE AND AGE OF OWN CHILDRE...,http://api.census.gov/data/2010/dec/sf1/groups...
3,P38C,FAMILY TYPE BY PRESENCE AND AGE OF OWN CHILDRE...,http://api.census.gov/data/2010/dec/sf1/groups...
4,P38I,FAMILY TYPE BY PRESENCE AND AGE OF OWN CHILDRE...,http://api.census.gov/data/2010/dec/sf1/groups...
...,...,...,...
328,PCT22B,GROUP QUARTERS POPULATION BY SEX BY GROUP QUA...,http://api.census.gov/data/2010/dec/sf1/groups...
329,PCT22G,GROUP QUARTERS POPULATION BY SEX BY GROUP QUA...,http://api.census.gov/data/2010/dec/sf1/groups...
330,PCT22H,GROUP QUARTERS POPULATION BY SEX BY GROUP QUA...,http://api.census.gov/data/2010/dec/sf1/groups...
331,PCT22E,GROUP QUARTERS POPULATION BY SEX BY GROUP QUA...,http://api.census.gov/data/2010/dec/sf1/groups...
